In [1]:
import json
import os
import random
import pandas as pd
import re
import requests
import selenium

In [ ]:
driver.get(url) # 打开url网页 比如 driver.get("http://www.baidu.com")

In [4]:
with open(f"data/wine_cet4_word.sql", 'r', encoding='utf-8') as file:
    words = file.read()

In [11]:
import re

def parse_sql_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        sql_content = file.read()
        
    # Regex to match the INSERT statements and capture the values
    pattern = re.compile(
        r"INSERT INTO `wine_cet4_word` VALUES \((\d+), '([^']*)', '([^']*)', '([^']*)', '([^']*)', '([^']*)', '([^']*)'\);"
    )

    matches = pattern.findall(sql_content)

    data = []
    for match in matches:
        entry = {
            'id': int(match[0]),
            'cet4_word': match[1],
            'cet4_phonetic': match[2],
            'cet4_translate': match[3],
            'cet4_distortion': match[4],
            'cet4_phrase': match[5],
            'cet4_samples': match[6]
        }
        data.append(entry)
    
    return data

# File path to the SQL file
file_path = 'data/wine_cet4_word.sql'
parsed_data = parse_sql_file(file_path)
with open("data/words_cet4.json", 'w', encoding='utf-8') as json_file:
    json.dump(parsed_data, json_file, ensure_ascii=False, indent=4)


In [19]:
word_list = []
for w in parsed_data:
    word = w["cet4_word"]
    cet4_translate = w["cet4_translate"].replace("，",",")
    if len( re.findall("n\.",cet4_translate))>=2:
        print(w)            
    cet4_phonetic = w["cet4_phonetic"]
    word_list.append(rf"{word} {cet4_translate}")
with open("data/words_cet4.txt","w") as f:
    f.write("\n".join(word_list))        

{'id': 5, 'cet4_word': 'tank', 'cet4_phonetic': '英:[tæŋk]\\n美:[tæŋk]', 'cet4_translate': 'n. 坦克；水槽；池塘\\nvt. 把…贮放在柜内；打败\\nvi. 乘坦克行进\\nn. (Tank)人名；(德、土、阿塞、土库、挪)汤克', 'cet4_distortion': '[ 复数 tanks 过去式 tanked 过去分词 tanked 现在分词 tanking 第三人称单数 tanks ]', 'cet4_phrase': '', 'cet4_samples': ''}
{'id': 10, 'cet4_word': 'cloud', 'cet4_phonetic': '英:[klaʊd]\\n美:[klaʊd]', 'cet4_translate': 'n. 云；阴云；云状物；一大群；黑斑\\nvt. 使混乱；以云遮蔽；使忧郁；玷污\\nvi. 阴沉；乌云密布\\nn. （Cloud） 克劳德（人名）', 'cet4_distortion': '', 'cet4_phrase': 'Red Cloud\\n红云;火烧云;赤芸;红色的云\\nCloud Storage\\n云端储存;云存储;云储存\\ncloud cover\\n云量;云层;云蔽\\n[气象]\\n私有云;私密之云;私人云\\nprivate cloud\\n荚状云;笠云\\nlenticular cloud\\n积云;积状云;积云\\n[气象]\\n云技术;云端科技;云端技术;云科技\\ncumulus cloud\\n云应用;云运用;云操纵', 'cet4_samples': '\\"Cloud Street\\" is already in its third printing.\\n《云街》已经是第3次印刷了。\\n《柯林斯英汉双解大词典》\\nThe moon was fuller than the night before, but the light was diffused by cloud.\\n月亮比头一天晚上更圆，但因云层遮掩而月光朦胧。\\n《牛津词典》\\nThe Sun, Moon, and the planets formed from a huge spinning gas

In [75]:
uppercase_letters = [chr(i) for i in range(65, 91)]
word_list = []
for i in uppercase_letters:
    print(i)
    with open(f"data/A_Z/{i}.json", 'r', encoding='utf-8') as file:
        words = json.load(file)
        for w in words:
            word = w["word"]
            mean = w["mean"].replace("，",",")
            if len( re.findall("n\.",mean))>=2:
                print(w)            
            phonetic_symbol = w["phonetic_symbol"]
            word_list.append(rf"{word} {mean}")
with open("data/words_3000.txt","w") as f:
    f.write("\n".join(word_list))        

A
B
C
D
{'initial': 'D', 'mean': 'n.指导者;理事;导演dirt[də:t]n.尘,土;污物,污垢', 'phonetic_symbol': '[di’rektə，də’rektə(r)，daɪ-]', 'word': 'director'}
E
F
G
H
I
J
{'initial': 'J', 'mean': 'n.旅行，旅程n.旅行', 'phonetic_symbol': '/‘dʒə:ni/', 'word': 'journey'}
K
L
{'initial': 'L', 'mean': 'n.表，目录n.列举', 'phonetic_symbol': '/list/', 'word': 'list'}
M
N
O
P
{'initial': 'P', 'mean': 'n.&vt.&n.轻拍', 'phonetic_symbol': '/pæt/', 'word': 'pat'}
{'initial': 'P', 'mean': 'n.针，饰针n.别住', 'phonetic_symbol': '/pin/', 'word': 'pin'}
{'initial': 'P', 'mean': 'n.衣袋n.袖珍的', 'phonetic_symbol': '/‘pɔkit/', 'word': 'pocket'}
Q
R
S
{'initial': 'S', 'mean': 'n.符号;招牌n.签名', 'phonetic_symbol': '/sain/', 'word': 'sign'}
T
U
V
W
X
Y
Z


In [24]:
import requests
from bs4 import BeautifulSoup
import json
user_agent = 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Mobile Safari/537.36 Edg/126.0.0.0'
header = {"User-Agent":user_agent,}
url = "https://zhenti.burningvocabulary.cn/cet4"
response = requests.get(url,headers=header)
response.encoding = 'utf-8'
soup = BeautifulSoup(response.text, 'html.parser')
html_content = str(soup)

# with open('output.html', 'w', encoding='utf-8') as file:
#     file.write(html_content)

In [25]:
pattern = re.compile(r'<a class="link-primary" href="([^"]+)"')
urls = pattern.findall(html_content)

In [93]:
root = r"https://zhenti.burningvocabulary.com"
root+urls[-1]
pdf_url = root+urls[5]


In [94]:
import requests
from selenium import webdriver
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

edge_driver_path = r'C:\src\application\edge_drive\msedgedriver.exe'
driver = webdriver.Edge()
driver.get(pdf_url)

In [100]:
response = requests.get(pdf_url,headers=header)

In [50]:
button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div[1]/div[2]/div[3]/div/div[1]/div[2]/button[5]'))
    )
button.click()

In [109]:
soup = BeautifulSoup(response.text, 'html.parser')

In [104]:
soup.find('查答案')

In [112]:
soup.find('div', class_='outerContainer')

In [106]:
soup

<!DOCTYPE html>
<html data-path="/cet4/2023-06/03" dir="ltr" lang="en"><head><meta charset="utf-8"/><meta content="width=device-width, initial-scale=1, maximum-scale=1" name="viewport"/><meta content="notranslate" name="google"/><meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/><title>2023年6月英语四级真题(第3套) | 英语真题在线</title><link href="/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/>
<link href="/favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/><meta content="yes" name="apple-mobile-web-app-capable"/><meta content="#ffffff" name="apple-mobile-web-app-status-bar-style"/><link href="/apple-touch-icon.png" rel="apple-touch-icon"/><meta content="2023年6月英语四级真题(第3套)，下载真题PDF，答案、听力、训练进度、生词本、在线训练。" name="description"/><meta content="2023年6月四级真题,第三套,2023四级真题,四级真题2023,2023四级真题听力,四级真题下载,四级真题听力,四级考试真题,四级考试真题" name="keywords"/><meta content="英语真题在线" property="og:site_name"/><meta content="2023年6月英语四级真题(第3套)" property="og:title"/><meta content="2023年6月英语四级真题(第3套)，下